# ETL Project
<ul>
    <li>UofMN Data Visualization and Analytics Bootcamp</li>
    <li>Week 13 | ETL Project</li>
    <li>Created by: Stephanie Hartje, Chris Howard</li>
    <li>05/18/2019</li>
</ul>

### Project Description and Purpose
<p>This project extracts(E) data from multiple sources, uses the Python Pandas module to transform(T) the data into 
    useful tables, which can then be mapped and loaded(L) into a SQL database. There is no direct analysis done on
    the data for the project, but the intention is to have a usable database for a theoretical analysis at the end of 
    the process.</p>
<p>Our theoretical analysis is looking at any (albeit spurious) correlation between solar eclipses, ufo sightings, and 
    multiple natural disasters including hurricanes and volcanoes. Each event type has been given its own table 
    in the database with a minimum of event date, some form of ID, and location (including latitude and longitude where
    available). All dates have been separated into 'year' 'month' 'day' columns so that events can be easily 
    compared by date for clusters around certain months as well as by year and location.</p>
<p>The sql code for our database can be found in our repository, or opened directly into a new Jupyter window <a href='../edit/disaster_etl.sql'>using this link</a> if this notebook is being run locally within a copy of the repository.</p>
<p>Extract:</p>
     <p>We extracted the following:</p>
        <ol><li>Solar Eclipses</li>
            <ul><li>From: https://data.world/nasa/five-millennium-catalog-of-solar-eclipses-detailed</li>
                <li>Original Format: CSV</li></ul>
        <li>UFO Sightings</li>
            <ul><li>From: https://en.wikipedia.org/wiki/List_of_reported_UFO_sightings</li>
                <li>Original Format: 2 HTML tables, one for 19th century and one for 20th century</li></ul>
        <li>Hurricanes</li>
            <ul><li>From: https://www.kaggle.com/noaa/hurricane-database</li>
                <li>Original Format: Two CSV files, one for Atlantic and one for Pacific Storms</li></ul>
        <li>Volcanoes:</li>
            <ul><li>From:  https://data.world/dhs/historical-significant</li>
                <li>Original Format: CSV</li></ul></ol>
<p>Transform:</p>
    <p>We performed the following transformation steps for each data set respectively:</p>
        <ol><li>Solar Eclipses</li>
            <ul><li></li></ul>
        <li>UFO Sightings</li>
            <ul><li>remove label row from 20th century table and combine into one table</li>
                <li>use first row as header and re-index</li>
                <li>separate year, month, and date into separate columns</li>
                <li>select the columns to keep</li></ul>
        <li>Hurricanes</li>
            <ul><li>combine Atlantic and Pacific tables into one table</li>
            <li>convert date column to string and separate year, month, and day into separate columns</li>
            <li>select the columns to keep</li>
            <li>select the rows corresponding to Hurricane data</li>
            <li>keep only the first observation related to a particular Hurricane</li>
            <li>convert latitude and longitude to format consistent with other tables</li></ul>
        <li>Volcanoes</li>
            <ul><li>select columns to keep</li>
                <li>rename columns</li></ul></ol>
<p>Load:</p>
    <p>We created a database in MySQL (disaster_etl) and created a table for each dataset.  We then loaded the data into MySQL using sqlalchemy.</p>
    

In [ ]:
# imports
import pandas as pd
import numpy as np
import requests
from sqlalchemy import create_engine
import config

In [ ]:
## Chris Extract/Transform below


In [ ]:
# ufo data from wikipedia, data from 19th & 20th 
ufo_url = 'https://en.wikipedia.org/wiki/List_of_reported_UFO_sightings'
ufo_df_19th = pd.read_html(ufo_url)[5]
ufo_df_20th = pd.read_html(ufo_url)[6]

# remove label row from 20th century data
ufo_df_20th = ufo_df_20th.drop(0)

# combine tables into single dataframe
ufo_df = ufo_df_19th.append(ufo_df_20th, ignore_index=True) 

# use first row as column headers, then reindex removing top row
ufo_df.columns = ufo_df.iloc[0]
ufo_df = ufo_df.reindex(ufo_df.index.drop(0))

# create loop to extract year/month/day from formatting
dates = ufo_df['Date']
year = []
month = []
day = []
for date in dates:
    date = date.strip('s')
    date = date.split('-')
    year.append(date[0])
    if len(date) > 1:
        month.append(date[1])
    else:
        month.append(None)
    if len(date) > 2:
        day.append(date[2])
    else:
        day.append(None)

# insert 'Year' 'Month' 'Day' columns into the dataframe
ufo_df.insert(loc=0, column='Year', value=year)
ufo_df.insert(loc=1, column='Month', value=month)
ufo_df.insert(loc=2, column='Day', value=day)
ufo_df_clean = ufo_df[['Year', 'Month', 'Day', 'Date', 'Name', 'Country', 'Description']].copy()
ufo_df_clean


In [ ]:
# Eclipse data, first needed manual cleaning in .csv files to remove extra columns from random rows
eclipse_1900 = pd.read_csv('Data/1901-2000.csv', index_col=False)
eclipse_2000 = pd.read_csv('Data/2001-2100.csv', index_col=False)
eclipse_df = eclipse_1900.append(eclipse_2000)

eclipse_df_clean = eclipse_df[['Catalog Number', 'Calendar Year', 'Calendar Month', 'Calendar Day', 'Ecl. Type',
                              u'Lat \N{DEGREE SIGN}', u'Long \N{DEGREE SIGN}']]

eclipse_df_clean = eclipse_df_clean.rename(columns={'Catalog Number':'catalog_number', 
                                 'Calendar Year': 'year', 
                                 'Calendar Month': 'month_old', 
                                 'Calendar Day': 'day', 
                                 'Ecl. Type': 'eclipse_type', 
                                 u'Lat \N{DEGREE SIGN}': 'latitude_old', 
                                 u'Long \N{DEGREE SIGN}': 'longitude_old'})

latitude = eclipse_df_clean['latitude_old']
new_lat = []
for lat in latitude:
    
    if lat[-1] == 'S':
        lat = lat[:-1]
        lat = '-' + ''.join(lat)
    else:
        lat = lat[:-1]
        lat = ''.join(lat)
    new_lat.append(lat)

longitude = eclipse_df_clean['longitude_old']
new_lon = []
for lon in longitude:
    if lon[-1] == 'W':
        lon = lon[:-1]
        lon = '-' + ''.join(lon)
    else:
        lon = lon[:-1]
        lon = ''.join(lon)
    new_lon.append(lon)

eclipse_df_clean['latitude'] = new_lat
eclipse_df_clean['longitude'] = new_lon
eclipse_df_clean = eclipse_df_clean.drop(columns=['latitude_old', 'longitude_old'])

months = eclipse_df_clean['month_old']
months_new = []
def months_to_numbers(argument): 
    switcher = { 
        'Jan': '01', 
        'Feb': '02', 
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    } 
    return switcher.get(argument, "nothing") 

for month in months:
    month = months_to_numbers(month)
    months_new.append(month)
    
eclipse_df_clean.insert(loc=2, column='month', value=months_new)
eclipse_df_clean = eclipse_df_clean.drop(columns=['month_old'])
eclipse_df_clean.head()

In [ ]:
## Stephanie Extract/Transform below

# Extract CSVs into DataFrames
    AtlanticStorms from https://www.kaggle.com/noaa/hurricane-database
        - Each date has up to 5 observations per day (but not all days have 5)
        - Older data appears to use -999 from wind pressure and speed instead of something like NA
        - ID: AL = Atlantic, XX = number storm for year, YYYY = year
    PacificStorms from https://www.kaggle.com/noaa/hurricane-database
        - Each date has up to 5 observations per day (but not all days have 5)
        - Older data appears to use -999 from wind pressure and speed instead of something like NA
        - ID: EP = Pacific, XX = number storm for year, YYYY = year
    Volcanoes from https://data.world/dhs/historical-significant

In [ ]:
#Extract Atlantic Storm Data

AtlanticStorm = "Data/Atlantic_Storms.csv"
AtlanticStorm_df = pd.read_csv(AtlanticStorm)
AtlanticStorm_df.head()

In [ ]:
#Extract Pacific Storm Data

PacificStorm = "Data/Pacific_Storms.csv"
PacificStorm_df = pd.read_csv(PacificStorm)
PacificStorm_df.head()

In [ ]:
#Transform Hurricane Data
# Combine Atlantic and Pacific Storm Data

AtlPacStorms = [AtlanticStorm_df, PacificStorm_df]
AtlPacStorms_df = pd.concat(AtlPacStorms).reset_index(drop=True)
AtlPacStorms_df.head()

In [ ]:
# Check that Pacific Storms are included in combined df

AtlPacStorms_df.loc[AtlPacStorms_df['ID'] == "EP011949"]

In [ ]:
AtlPacStorms_df.dtypes

In [ ]:
# Adjust date format

# make string version of original Date column, call it 'col'
AtlPacStorms_df['col'] = AtlPacStorms_df['Date'].apply(str)

# make the new columns using string indexing
AtlPacStorms_df['Year'] = AtlPacStorms_df['col'].str[0:4]
AtlPacStorms_df['Month'] = AtlPacStorms_df['col'].str[4:6]
AtlPacStorms_df['Day'] = AtlPacStorms_df['col'].str[6:8]

# get rid of the extra variable (if you want)
AtlPacStorms_df.drop('col', axis=1, inplace=True)

#check result
AtlPacStorms_df.head()

In [ ]:
#Select columns to keep

AtlPacStorms_df = AtlPacStorms_df[["Year", "Month", "Day", "ID", "Status", "Time", "Latitude", "Longitude"]]
AtlPacStorms_df.head()


In [ ]:
AtlPacStorms_df["Status"] = AtlPacStorms_df['Status'].astype(str)
AtlPacStorms_df.dtypes

In [ ]:
# We are only interested in Hurricanes to only keep rows with Status = HU

AtlPacStorms_df = AtlPacStorms_df.loc[AtlPacStorms_df["Status"] == " HU"]
AtlPacStorms_df.head()

In [ ]:
# Keep only the first observation of each unique ID

Hurricane_df = AtlPacStorms_df.drop_duplicates(subset=[AtlPacStorms_df.columns[3]], keep = "first")
Hurricane_df.head()

In [ ]:
# Drop Status and Time columns

Hurricane_df = Hurricane_df[["Year", "Month", "Day", "ID", "Latitude", "Longitude"]]
Hurricane_df = Hurricane_df.reset_index(drop = True)
Hurricane_df.head()


In [ ]:
#Re-format Latitude and Longitude to align with format in other tables
#rename columns
Hurricane_df.columns = ["Year", "Month", "Day", "ID", "latitude_old", "longitude_old"]
Hurricane_df.head()

In [ ]:
#Change format
latitude = Hurricane_df['latitude_old']
new_lat = []
for lat in latitude:

   if lat[-1] == 'S':
       lat = lat[:-1]
       lat = '-' + ''.join(lat)
   else:
       lat = lat[:-1]
       lat = ''.join(lat)
   new_lat.append(lat)

longitude = Hurricane_df['longitude_old']
new_lon = []
for lon in longitude:
   if lon[-1] == 'W':
       lon = lon[:-1]
       lon = '-' + ''.join(lon)
   else:
       lon = lon[:-1]
       lon = ''.join(lon)
   new_lon.append(lon)

In [ ]:
#add new lists to dataframe
Hurricane_df['Latitude'] = new_lat
Hurricane_df['Longitude'] = new_lon
Hurricane_df.head()

In [ ]:
#drop old columns
Hurricane_df = Hurricane_df[['Year','Month','Day','ID','Latitude','Longitude']]
Hurricane_df.head()

In [ ]:
#Extract Volcano Data

Volcano = "Data/Historical_Significant_Volcanic_Eruption_Locations.csv"
Volcano_df = pd.read_csv(Volcano)
Volcano_df.head()

In [ ]:
#Transform Volcano Data
#Select columns

Volcano_df = Volcano_df[["YEAR", "MO", "DAY", "VOL_ID", "NAME", "LOCATION", "LATITUDE", "LONGITUDE"]]
Volcano_df.head()

In [ ]:
#Rename Columns

Volcano_df.columns = ['Year', 'Month', 'Day', 'Volcano_ID', 'Volcano_Name', 'Location', 'Latitude', 'Longitude']
Volcano_df.head()

In [ ]:
## Chris Load below

In [ ]:
conn = f"{config.username}:{config.password}@127.0.0.1/disaster_etl"
engine = create_engine(f'mysql+pymysql://{conn}')

In [ ]:
engine.table_names()

In [ ]:
ufo_df_clean.to_sql(name='ufo_sightings', con=engine, if_exists='append', index=False)

In [ ]:
eclipse_df_clean.to_sql(name='eclipse_event', con=engine, if_exists='append', index=False)

In [ ]:
## Stephanie Load below

In [ ]:
conn = f"{config.username}:{config.password}@127.0.0.1/disaster_etl"
engine = create_engine(f'mysql+pymysql://{conn}')

In [ ]:
engine.table_names()

In [ ]:
Volcano_df.to_sql(name='volcano_eruptions', con=engine, if_exists='append', index=False)

In [ ]:
Hurricane_df.to_sql(name='hurricanes', con=engine, if_exists='append', index=False)